In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
data = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [12]:
data.head()

In [13]:
data.shape

In [14]:
data.info()

In [16]:
data['review'][0]

# Text Preprocessing

1. Remove html tag
2. Remove special characters
3. Converting everything to lower case
4. Remove Stopwords (eg and, in, me, who)
5. Stemming
6. Convert sentiments from posstive to 1 and negative to 0

In [17]:
# Function to remove html tags
import re
def clean_tags(text):
    clean = re.compile("<.*?>")
    return re.sub(clean, '', text)

In [18]:
data['review'] = data['review'].apply(clean_tags)

In [22]:
# Function to remove special characters
def remove_special_char(text):
    x=''
    for i in text:
        if i.isalnum():    
            x = x + i
        else:
            x = x + ' '
    return x    

In [23]:
data['review'] = data['review'].apply(remove_special_char)

In [26]:
# Function to convert into lower case
def to_lower(text):
    return text.lower()   

In [27]:
data['review'] = data['review'].apply(to_lower)

In [37]:
# Function to remove Stopwords
import nltk
from nltk.corpus import stopwords

def remove_stopwords(text):
    x=[]
    stopWords = stopwords.words('english')
    for i in text.split():
        if i not in stopWords:
            x.append(i)
    y = x[:]
    x.clear()
    return y

In [38]:
data['review'] = data['review'].apply(remove_stopwords)

In [33]:
# Calculating stopwords
len(stopwords.words('english'))

In [41]:
# Performing Stemming
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

y=[]
def perform_stemmer(text):
    for i in text:
        y.append(ps.stem(i))
    z = y[:]
    y.clear()
    return z

In [42]:
data['review'] = data['review'].apply(perform_stemmer)

In [43]:
# join back
def join_back(list_input):
    return " ".join(list_input)

In [44]:
data['review'] = data['review'].apply(join_back)

In [48]:
# transforming col values
data['sentiment'] = data['sentiment'].rename({"positive":1, "negative":0})

In [49]:
data.head()

# Model Training

In [52]:
X = data.iloc[:,0:1].values

In [53]:
X.shape

In [98]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000)

In [99]:
X = cv.fit_transform(data['review']).toarray()

In [100]:
X.shape

In [101]:
y = data.iloc[:,-1].values

In [102]:
# Train test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [103]:
print(X_train.shape)
print(y_train.shape)

In [104]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

In [105]:
clf1 = GaussianNB()
clf2 = MultinomialNB()
clf3 = BernoulliNB()

In [106]:
#For Guassian
clf1.fit(X_train, y_train)

In [107]:
#For Multinomial
clf2.fit(X_train, y_train)

In [108]:
#For Bernoulli
clf3.fit(X_train, y_train)

In [109]:
y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)

In [110]:
from sklearn.metrics import accuracy_score
print("Gaussian: ", accuracy_score(y_test, y_pred1))
print("Multinomial: ", accuracy_score(y_test, y_pred2))
print("Bernoulli: ", accuracy_score(y_test, y_pred3))